In [2]:
import pandas as pd
import os
import pickle

In [3]:
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Move one level up

# 1. Choose the phenotype

In [4]:
#phenotype = "YPD_doublingtime"
phenotype = "YPDCUSO410MM_40h"

# 2. Load the data

In [5]:
x_matrix_path = os.path.join(data_dir, f"data/X_matrix_{phenotype}.pkl")
important_proteins_path = os.path.join(data_dir, f"results/combined_importance_ranking.csv")

X_matrix = pd.read_pickle(x_matrix_path)
important_proteins = pd.read_csv(important_proteins_path)

# 3. Modify the columns names of the X_matrix

In [6]:
columns = X_matrix.columns

# Function to add the suffix _CNV if the name is not in the Protein_ID_index_AA_AA format
def transform_protein_name(protein_name):
    if "_" not in protein_name:  # Checks if it is a CNV 
        return f"{protein_name}_CNV"
    return protein_name

new_columns = [transform_protein_name(col) for col in columns]
X_matrix.columns = new_columns

# 4. Create the X_matrix (mutations from important proteins)

In [7]:
important_features = important_proteins['Feature_ID'].tolist()
filtered_columns = ['Yeast_ID'] + [col for col in X_matrix.columns if any(protein in col for protein in important_features)]
X_matrix_restricted = X_matrix[filtered_columns]

X_matrix_restricted.to_csv(os.path.join(data_dir, f"data/X_matrix_restricted_{phenotype}.csv"), index=False)
X_matrix_restricted.to_pickle(os.path.join(data_dir, f"data/X_matrix_restricted_{phenotype}.pkl"))
